# Evaluate the classifier's transferability on a tiled dataset from Bern cohorts

In [ ]:
import os
import functools

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
from tqdm import tqdm

import Abed_utils

## Load and build model

In [ ]:
patch_size = 8
batch_size = 8
im_size = 224
path_to_backbone = './ckpts/dino_deitsmall8_pretrain.pth'
path_to_classifier = './ckpts/classifier_K19_CE_100ep_one_layer.pt'

transform = functools.partial(Abed_utils.normalize_input, im_size=im_size, patch_size=patch_size)

In [ ]:
backbone = Abed_utils.get_vit(patch_size, path_to_backbone)

classifier = Abed_utils.ClassificationHead(pretrained_path=path_to_classifier)

model = nn.Sequential(backbone, classifier)
model.eval()

In [ ]:
if Abed_utils.BERN_TILES_ROOT is None:
    raise RuntimeError('labelled Bern tiles not available on this machine, run on the dataserver')

acc = 0
f1 = 0

wsis = os.listdir(Abed_utils.BERN_TILES_ROOT)
with torch.no_grad():
    for wsi in wsis:
        ds = Abed_utils.BernTilesLabelDataset(wsi, transform=transform)
        data = DataLoader(ds, batch_size=batch_size)
        preds = np.empty(len(ds))
        targets = np.empty_like(preds)
        for i, (x, y) in enumerate(tqdm(data)):
            idx = slice(i*batch_size,min((i+1)*batch_size, preds.shape[0]))
            preds[idx] = model(x).cpu().numpy()
            targets[idx] = y.cpu.numpy()

